In [1]:
import numpy as np
import pandas as pd
import tushare as ts

In [36]:
ts.set_token('4c1d16a895e4c954adc8d2a436f2b21dd4ccc514f0c5a192edaa953b')
pro = ts.pro_api()
df = pro.index_weight(index_code='000905.SH', start_date='20200101', end_date='20200131')

In [37]:
df.empty

True

In [4]:
from pymongo import MongoClient, ASCENDING, DESCENDING
mc = MongoClient("mongodb://124.70.183.208:27017/", username='root', password='qiuqiu78')
db = mc['testdb']  # 数据库

In [5]:
cl_zz500 = db['stock_zz500']
cl_zz500.create_index([('trade_date', ASCENDING), ('con_code', ASCENDING)], unique=True)
cl_zz500.insert_many(json.loads(df.T.to_json()).values())

NameError: name 'json' is not defined

In [ ]:
a = df.loc[:, 'con_code'].apply(lambda x : x.replace('.', '_'))

In [ ]:
df.loc[:, 'ts_code'] = a

In [ ]:
df.drop(columns=['con_code'], inplace=True)

In [38]:
df.head()

,index_code,con_code,trade_date,weight


In [48]:
df1 = pro.index_daily(ts_code='000905.SH', start_date='20150101', end_date='20200817')

In [50]:
df1.head()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000905.SH,20200817,6744.5468,6646.5418,6752.3176,6614.1849,6623.5633,120.9835,1.8266,201407357.0,207540650.7
1,000905.SH,20200814,6623.5633,6542.6032,6629.5058,6506.0266,6552.6567,70.9066,1.0821,156875988.0,161300612.5
2,000905.SH,20200813,6552.6567,6552.3695,6592.1163,6518.4229,6528.3999,24.2568,0.3716,162060515.0,166812245.2
3,000905.SH,20200812,6528.3999,6580.0058,6599.6573,6395.0060,6600.8472,-72.4473,-1.0975,177889045.0,197969436.4
4,000905.SH,20200811,6600.8472,6727.7692,6756.8451,6594.9793,6725.2956,-124.4484,-1.8505,180845771.0,203427777.8


In [9]:
df1.tail()

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
1365,000905.SH,20150109,5409.058,5424.501,5497.689,5409.058,5444.037,-34.979,-0.6425,90640808.0,1.053903e+08
1366,000905.SH,20150108,5444.037,5482.394,5497.744,5418.011,5488.242,-44.205,-0.8054,95634617.0,1.085210e+08
1367,000905.SH,20150107,5488.242,5471.402,5492.825,5436.918,5479.864,8.378,0.1529,96050494.0,1.117871e+08
1368,000905.SH,20150106,5479.864,5389.903,5482.124,5354.338,5417.017,62.847,1.1602,123754496.0,1.403281e+08
1369,000905.SH,20150105,5417.017,5338.752,5427.231,5314.339,5322.714,94.303,1.7717,128787603.0,1.393430e+08


In [15]:
import numpy
import talib

close = np.array([1.0,2,3,4,5])

In [55]:
talib.SMA(close, 5)

array([nan, nan, nan, nan,  3.])

In [46]:
df1.loc[:,'trade_date'] = '111'

In [47]:
df1

,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,000905.SH,111,6744.5468,6646.5418,6752.3176,6614.1849,6623.5633,120.9835,1.8266,201407357.0,2.075407e+08
1,000905.SH,111,6623.5633,6542.6032,6629.5058,6506.0266,6552.6567,70.9066,1.0821,156875988.0,1.613006e+08
2,000905.SH,111,6552.6567,6552.3695,6592.1163,6518.4229,6528.3999,24.2568,0.3716,162060515.0,1.668122e+08
3,000905.SH,111,6528.3999,6580.0058,6599.6573,6395.0060,6600.8472,-72.4473,-1.0975,177889045.0,1.979694e+08
4,000905.SH,111,6600.8472,6727.7692,6756.8451,6594.9793,6725.2956,-124.4484,-1.8505,180845771.0,2.034278e+08
5,000905.SH,111,6725.2956,6666.5601,6768.8660,6651.7421,6688.8822,36.4134,0.5444,175353227.0,2.103909e+08
6,000905.SH,111,6688.8822,6743.2907,6755.5280,6578.6672,6770.3016,-81.4194,-1.2026,198128214.0,2.279767e+08
7,000905.SH,111,6770.3016,6780.3265,6792.2759,6656.2782,6772.8422,-2.5406,-0.0375,205229842.0,2.409289e+08
8,000905.SH,111,6772.8422,6696.6870,6790.0080,6653.2772,6697.7071,75.1351,1.1218,184843816.0,2.179942e+08
9,000905.SH,111,6697.7071,6758.4937,6764.9049,6672.8940,6739.8085,-42.1014,-0.6247,209478936.0,2.450848e+08


In [79]:
pd.Series([1,1,1,1,1])

0    1
1    1
2    1
3    1
4    1
dtype: int64

In [82]:
a=  pd.Series([1,1,1,1,1])
dft.loc[:, 'tt'] = a

In [83]:
dft

,index_code,trade_date,weight,ts_code,tt
0,000905.SH,20180928,0.261,603899_SH,1
1,000905.SH,20180928,0.108,603888_SH,1
2,000905.SH,20180928,0.218,603885_SH,1
3,000905.SH,20180928,0.163,603883_SH,1
4,000905.SH,20180928,0.037,603877_SH,1


In [85]:
pro.index_daily??